# Get data from Johns Hopkins github repository
https://github.com/CSSEGISandData/COVID-19.


In [1]:
!git clone https://github.com/CSSEGISandData/COVID-19.git

Cloning into 'COVID-19'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 20834 (delta 10), reused 16 (delta 9), pack-reused 20813
Receiving objects: 100% (20834/20834), 86.63 MiB | 28.99 MiB/s, done.
Resolving deltas: 100% (11029/11029), done.


In [2]:
%cd COVID-19
!ls

/content/COVID-19
archived_data  csse_covid_19_data  README.md  who_covid_19_situation_reports


# Pascal's COVID-19 scripts

In [3]:
#@title
"""Plot COVID-19 time series data from Johns Hopkins University.

This script makes a few plots of COVID-19 cases from the data in Johns Hopkins'
github repo https://github.com/CSSEGISandData/COVID-19.


How to use this script:

0. Install matplotlib, numpy, pandas.

1. Clone the github repo to download the repo

  git clone https://github.com/CSSEGISandData/COVID-19.git

Later you can `git pull` to update the data. The repo updates about twice a day.

2. Put this script inside the repo's root COVID-19 directory.

3. Run the script

  python3 plot_covid_19.py


Pascal Getreuer 2020
"""



"Plot COVID-19 time series data from Johns Hopkins University.\n\nThis script makes a few plots of COVID-19 cases from the data in Johns Hopkins'\ngithub repo https://github.com/CSSEGISandData/COVID-19.\n\n\nHow to use this script:\n\n0. Install matplotlib, numpy, pandas.\n\n1. Clone the github repo to download the repo\n\n  git clone https://github.com/CSSEGISandData/COVID-19.git\n\nLater you can `git pull` to update the data. The repo updates about twice a day.\n\n2. Put this script inside the repo's root COVID-19 directory.\n\n3. Run the script\n\n  python3 plot_covid_19.py\n\n\nPascal Getreuer 2020\n"

## Imports

In [0]:
from datetime import datetime
import glob
import os.path
import matplotlib.dates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Constants and definitions

In [0]:
# Glob of daily report CSV files.
COVID19_CSV_GLOB = 'csse_covid_19_data/csse_covid_19_daily_reports/*.csv'

# Map from US state code to state name.
# Used for normalizing some naming inconsistencies.
US_STATE_CODE_TO_NAME = {
  'AL': 'Alabama',
  'AK': 'Alaska',
  'AZ': 'Arizona',
  'AR': 'Arkansas', 
  'CA': 'California',
  'CO': 'Colorado',
  'CT': 'Connecticut',
  'DE': 'Delaware',
  'DC': 'D.C.',
  'FL': 'Florida',
  'GA': 'Georgia',
  'HI': 'Hawaii',
  'ID': 'Idaho',
  'IL': 'Illinois',
  'IN': 'Indiana',
  'IA': 'Iowa',
  'KS': 'Kansas',
  'KY': 'Kentucky',
  'LA': 'Louisiana',
  'ME': 'Maine',
  'MD': 'Maryland',
  'MA': 'Massachusetts',
  'MI': 'Michigan',
  'MN': 'Minnesota',
  'MS': 'Mississippi',
  'MO': 'Missouri',
  'MT': 'Montana',
  'NE': 'Nebraska',
  'NV': 'Nevada',
  'NH': 'New Hampshire',
  'NJ': 'New Jersey',
  'NM': 'New Mexico',
  'NY': 'New York',
  'NC': 'North Carolina',
  'ND': 'North Dakota',
  'OH': 'Ohio',
  'OK': 'Oklahoma',
  'OR': 'Oregon',
  'PA': 'Pennsylvania',
  'RI': 'Rhode Island',
  'SC': 'South Carolina',
  'SD': 'South Dakota',
  'TN': 'Tennessee',
  'TX': 'Texas',
  'UT': 'Utah',
  'VT': 'Vermont',
  'VA': 'Virginia',
  'WA': 'Washington',
  'WV': 'West Virginia',
  'WI': 'Wisconsin',
  'WY': 'Wyoming',
}

# Country population estimates for a few countries as of July 1, 2019, from
# https://en.wikipedia.org/wiki/List_of_countries_by_population_(United_Nations)
COUNTRY_POPULATION = {
  'China': 1433783686,
  'India': 1366417754,
  'US': 329064917,
  'Indonesia': 270625568,
  'Pakistan': 216565318,
  'Brazil': 211049527,
  'Nigeria': 200963599,
  'Bangladesh': 163046161,
  'Russia': 145872256,
  'Mexico': 127575529,
  'Japan': 126860301,
  'Ethiopia': 112078730,
  'Philippines': 108116615,
  'Egypt': 100388073,
  'Germany': 83517045,
  'United Kingdom': 67530172,
  'France': 65129728,
  'Iran': 82913906,
  'Italy': 60550075,
  'South Korea': 51225308,
  'Spain': 46736776,
  'Netherlands': 17097130,
  'Switzerland': 8591365,
}

# US state population estimates for a few states as of July 1, 2019, from
# https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population
US_STATE_POPULATION = {
  'California': 39512223,
  'Texas': 28995881,
  'Florida': 21477737,
  'New York': 19453561,
  'Pennsylvania': 12801989,
  'New Jersey': 8882190,
  'Virginia': 8535519,
  'Washington': 7614893,
  'Georgia': 10617423,
  'Illinois': 12671821,
  'Michigan': 9986857,
  'Alabama': 4903185,
  'Louisiana': 4648794,
}

PLOT_COLORS = plt.rcParams['axes.prop_cycle'].by_key()['color']

## Classes

In [0]:
class Dataset(object):
  """Wrapper around DataFrame with some convience methods."""

  def __init__(self):
    """Constructor. Loads COVID-19 data from JHU's daily report CSV files."""
    dfs, keys = [], []
    
    for csv_file in glob.glob(COVID19_CSV_GLOB):
      try:
        # Column names changed after March 23 2020.
        if os.path.basename(csv_file) >= '03-22-2020.csv':
          index_col = ['Country_Region', 'Province_State']
        else:
          index_col = ['Country/Region', 'Province/State']
        use_cols = set(index_col + ['Confirmed', 'Deaths', 'Recovered'])
        
        # Each CSV file represents one day.
        dfs.append(pd.read_csv(csv_file,
          index_col=index_col,
          usecols=lambda x: x in use_cols))
        date = os.path.splitext(os.path.basename(csv_file))[0]
        keys.append(date)
      except ValueError as e:
        raise ValueError('%s: %s' % (csv_file, e))
      
    keys = pd.to_datetime(keys)
    self.df = pd.concat(dfs, keys=keys)  # Concat days to make time series.

    def _normalize_index(date, country, province):
      country = country.strip()
      province = province.strip() if isinstance(province, str) else province
      # Normalize for some inconsistencies in naming.
      if country == 'Mainland China':
        country = 'China'
      elif country in ('Korea, South', 'Republic of Korea'):
        country = 'South Korea'
      elif country == 'Iran (Islamic Republic of)':
        country = 'Iran'
    
      if country == 'US':
        if ', ' in province: # Strip city or county name if present.
          state_code = province.rsplit(', ', 1)[-1][:2]
          if state_code in US_STATE_CODE_TO_NAME:
            province = US_STATE_CODE_TO_NAME[state_code]

      return country, province, date

    self.df.index = pd.MultiIndex.from_tuples([
        _normalize_index(*x) for x in self.df.index],
        names=('Country', 'Province', 'Date'))
    self.df.sort_index(inplace=True)
    self.df.fillna(value=0, inplace=True)  # Replace missing with zero.
    # Cast case counts to integer.
    for col in self.df.columns:
      self.df[col] = self.df[col].astype(int)
    
    self.df['Active'] = (
        self.df['Confirmed'] - self.df['Deaths'] - self.df['Recovered'])
    # self.df is a DataFrame with 3-level index ('Country', 'Province', 'Date')
    # and columns ('Confirmed', 'Deaths', 'Recovered', 'Active').
    
    # Get time series of country totals by summing up provinces.
    # self.df_countries has a 2-level index ('Country', 'Date').
    self.df_countries = self.df.groupby(level=(0, 2)).sum()
    # Get time series of US state totals.
    # self.df_us_states has a 2-level index ('Province', 'Date').
    self.df_us_states = self.df.loc['US'].groupby(level=(0, 1)).sum()
    
  def get_country(self, country):
    """Gets time series of total cases in `country`."""
    df_country = self.df_countries.loc[country]
    df_country.df_name = country
    return df_country
    
  def get_province(self, province, country):
    """Gets time series of total cases in `province`."""
    df_province = self.df.loc[(country, province)].groupby(level=0).sum()
    df_province.df_name = '%s, %s' % (province, country)
    return df_province

## Global methods

In [29]:
# Make a global Dataset object `d` for convenience.
d = Dataset()

def print_data_summaries(n_countries = -1, n_us_states = -1):
  # Print some summary info.
  data_date = d.get_country('US').index[-1].date()
  print('Data last updated on %s' % data_date)

  # Get latest data for the countries with the most current active cases.

  if n_countries == -1:
    world_summary = d.df_countries.xs(data_date, level=1)#.nlargest(7, 'Active')
  else:
    world_summary = d.df_countries.xs(data_date, level=1).nlargest(n_countries, 'Active')    

  active_per_million = []
  for country, active in world_summary['Active'].iteritems():
    if country in COUNTRY_POPULATION:
      active_per_million.append(active * 1e6 / COUNTRY_POPULATION[country])
    else:
      active_per_million.append(None)
  world_summary['Active per 1M people'] = active_per_million

  print('\nMost affected countries:')
  print(world_summary)
  
  # Get latest data for the US states with the most current active cases.
  if n_us_states == -1:
    us_summary = d.df_us_states.xs(data_date, level=1)#.nlargest(7, 'Active')
  else:
    us_summary = d.df_us_states.xs(data_date, level=1).nlargest(n_us_states, 'Active')

  active_per_million = []
  for country, active in us_summary['Active'].iteritems():
    if country in US_STATE_POPULATION:
      active_per_million.append(active * 1e6 / US_STATE_POPULATION[country])
    else:
      active_per_million.append(None)
  us_summary['Active per 1M people'] = active_per_million

  print('\nMost affected US states:')
  print(us_summary)

  
def date_to_plot_coords(x):
  """Map datetime64 values to plotting coordinates."""
  if isinstance(x, pd.Series):
    x = x.index.values
  return x.astype(float) / (1e9 * 24 * 60 * 60)
  
  
def predict_growth(series):
  """Given a series, extrapolate the next 3 days based on last 7 days."""
  last_week = series[-7:]
  delta_days = np.arange(4)

  p = np.polyfit(np.arange(-6, 1),
                 np.log(np.maximum(1e-8, last_week.values)), 1)
  c = np.exp(p[1])
  T = 0
  if p[0] > 0:
    T = 1/(np.log2(np.e) * p[0])
    note = ' (doubles in %.1f days)' % T
  else:
    note = ''
  
  prediction = pd.Series(
      np.exp(np.polyval(p, delta_days)),
      index=[last_week.index[-1] + np.timedelta64(d, 'D')
          for d in delta_days])
  # Return the predicted series, a text note, and model parameters (c, T).
  return prediction, note, (c, T)
  

def make_plot_with_predictions(df, ax=None, logy=False):
  """Make a plot including 3-day predictions as dotted lines.
  
  Each column of `df` is plotted as a line. Rows of `df` should be dates.
  """
  if ax is None:
    ax = plt.gca()
    
  markers = ['+', '.', '']
  
  for pred in (False, True):
    for i, country in enumerate(df.columns):
      series = df[country]
      prediction, note, _ = predict_growth(series)
      series = series.rename(index=(country + note))
      if not pred:
        series.plot(style='-', marker=markers[i % len(markers)],
                    color=PLOT_COLORS[i], ax=ax, logy=logy)
      else:
        plot_fun = ax.semilogy if logy else ax.plot
        plot_fun(date_to_plot_coords(prediction),
            prediction.values, ':', color=PLOT_COLORS[i])
    
  plt.legend()
  x_min = date_to_plot_coords(series)[0]
  x_max = date_to_plot_coords(prediction)[-1]
  plt.xlim(x_min, x_max)
  plt.grid(True)
  

def get_active_ts_for_countries(countries, per_million=False):
  df = d.df_countries['Active'].unstack(level=0)[countries].fillna(value=0)
  if per_million:
    for country in countries:
      df[country] *= 1e6 / COUNTRY_POPULATION[country]
  return df
  
  
def get_active_ts_for_us_states(states, per_million=False):
  df = d.df_us_states['Active'].unstack(level=0)[states].fillna(value=0)
  if per_million:
    for state in states:
      df[state] *= 1e6 / US_STATE_POPULATION[state]
  return df



ValueError: ignored

In [0]:
# most_affected_countries

# Alex's Altair Visualizations


In [0]:
import altair as alt
import pandas as pd

def getCountriesNLargest(n = 10, criteria = 'Confirmed'):

  # find the countries with the currently n largest cases
  data_date = d.get_country('US').index[-1].date()
  print('Data last updated on %s' % data_date)
  # Will return PD with index = 'Country' but data is only from data_date
  n_active = d.df_countries.xs(data_date, level=1).nlargest(n, criteria)
  # Use the country indices to obtain all the data
  n_countries = d.df_countries.loc[n_active.index]

  return n_countries

def getCountriesNordic():
  nordic_countries = d.df_countries.loc[['Denmark', 'Finland', 'Iceland', 'Norway', 'Sweden' ]]
  return nordic_countries



## Comparing curves for single country

In [0]:
def getCountryPlots(countries, index = 0):

  # Use reset_index() to convert 'date' index to column, since Altair
  # ignores index values: https://github.com/altair-viz/altair/issues/271
  data_country = countries.reset_index()

  #data_country = data_country.melt('Date', var_name="Type", value_name = "Count")

  countries = list(data_country['Country'].unique())

  input_dropdown = alt.binding_select(options=countries)
  selection = alt.selection_single(fields=['Country'], bind=input_dropdown, name='Select', init={'Country': countries[index]})

  country_chart = alt.Chart(data_country).transform_fold(
            ['Recovered', 'Active', 'Deaths', 'Confirmed'],
            as_= ['Type', 'Count']
          ).mark_line().encode( 
  #        y = alt.Y('Count:Q', axis=alt.Axis(format=',.2d', title=None)),
          y = alt.Y('Count:Q', ),

          x=alt.X('Date', axis=alt.Axis(format='%b %d')), #https://github.com/d3/d3-time-format#locale_format
          color='Type:N',
          opacity=alt.value(0.7)
          ).add_selection(
            selection
          ).transform_filter(
            selection          
          ).configure_axis(
              grid=False,
              ticks=False,
              title=None,
              domainColor='lightgray',
              labelFont = 'Roboto Condensed',
              
        ).configure_axisY(
              grid=True,
              ).configure_view(
              strokeWidth=0
          ).configure_area(
          ).configure_legend(
              labelFont = 'Roboto Condensed',
              title = None 
          ).interactive() 

  return country_chart

# getCountryPlots()

## Multi-plot, comparing countries

In [0]:
def getMultiPlots(countries):

  WIDTH = 600
  HEIGHT = WIDTH
  N_SUBCHARTS = 3
  WS = WIDTH/N_SUBCHARTS
  HS = HEIGHT/N_SUBCHARTS

  # Use reset_index() to convert 'date' index to column, since Altair
  # ignores index values: https://github.com/altair-viz/altair/issues/271
  data_country = countries.reset_index()
  selection = alt.selection_multi(fields=['Country'], bind='legend')

  # Create the base chart
  base = alt.Chart(data_country).mark_line().encode( 
          color='Country',
          opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
          tooltip = ['Date', 'Country', 'Active', 'Recovered', 'Deaths'],

          ).add_selection(
            selection
          ).interactive().properties(width = WS, height = HS)

  multi_chart = alt.vconcat(data=data_country)

  row = alt.hconcat()
  row |= base.encode(y=alt.Y('Recovered', scale = alt.Scale(type='linear')), x='Deaths').properties(title='Recovered vs. Deaths', width=WIDTH/2, height=HEIGHT/2)
  row |= base.encode(y='Confirmed', x='Deaths').properties(title='Confirmed vs. Deaths', width=WIDTH/2, height=HEIGHT/2)
  multi_chart &= row

  row = alt.hconcat()
  row |= base.encode(opacity=alt.condition(selection, alt.value(1), alt.value(0.2)), y=alt.Y('Confirmed', axis=alt.Axis(title='')), x=alt.X('Date', axis=alt.Axis(title=''))).properties(title='Confirmed')
#  row |= base.encode(opacity=alt.condition(selection, alt.value(1), alt.value(0.2)), y=alt.Y('Active', axis=alt.Axis(title='')), x=alt.X('Date', axis=alt.Axis(title=''))).properties(title='Active')
  row |= base.encode(y=alt.Y('Deaths', axis=alt.Axis(title='')), x=alt.X('Date', axis=alt.Axis(title=''))).properties(title='Deaths')
  row |= base.encode(y=alt.Y('Recovered', axis=alt.Axis(title='')), x=alt.X('Date', axis=alt.Axis(title=''))).properties(title='Recovered')
  multi_chart &= row


  #does work
  multi_chart = multi_chart.configure_view(
      strokeWidth=0
  ).configure_axis(
    grid=False,
    ticks=False,
    # title=None,
    domainColor='lightgray',
    labelFont = 'Roboto Condensed',
    titleFont = 'Roboto Condensed',
    titleFontWeight = 'normal',
  ).configure_axisY(
    grid=True,
  ).configure_title(
    font = 'Roboto Condensed',
    fontWeight = 'normal'
  ).configure_legend(
              labelFont = 'Roboto Condensed',
              title = None 
  )

  return multi_chart

# nordic = getMultiPlots(getCountriesNordic())
# nLargest = getMultiPlots(getCountriesNLargest())
# nordic
 


## Plot charts

In [58]:
nordic = getMultiPlots(getCountriesNordic())
nLargest = getMultiPlots(getCountriesNLargest())

nordic = getCountryPlots(getCountriesNordic(), 4)
nLargest = getCountryPlots(getCountriesNLargest())

nordic.display()
nLargest.display()

Data last updated on 2020-04-17
Data last updated on 2020-04-17


alt.Chart(...)

alt.Chart(...)

## Exporting plots

In [0]:
from google.colab import files

def enableExports():

  !pip install -q  altair_saver
  import altair_saver
  from altair_saver import save

  !pip install selenium
  !apt-get update
  !apt-get install chromium-chromedriver


def exportPlots():

  nordicMulti = getMultiPlots(getCountriesNordic())
  nLargestMulti = getMultiPlots(getCountriesNLargest())

  nordic_countries = ['Denmark', 'Finland', 'Iceland', 'Norway', 'Sweden' ]

  n = getCountriesNordic()
  nordicCountry = [ getCountryPlots(n, 0), getCountryPlots(n, 1), getCountryPlots(n, 2), getCountryPlots(n, 3), getCountryPlots(n, 4) ]
  nLargestCountry = getCountryPlots(getCountriesNLargest())

  types = [ 'json', 'svg', 'png', 'html' ]

  !rm *.json *.svg *.png *.html

  for suffix in types:
    nordicMulti.save('nordic_multi_plots.' + suffix)
    nLargestMulti.save('nlargest_multi_plots.' + suffix)

    for i, c in enumerate(nordicCountry):
      c.save(nordic_countries[i].lower() + '_country_plots.' + suffix)

    nLargestCountry.save('nlargest_country_plots.' + suffix)

  !ls

  !rm plots.zip
  !zip plots.zip *.json *.svg *.png *.html
  files.download('plots.zip')


In [64]:
nordic_countries = ['Denmark', 'Finland', 'Iceland', 'Norway', 'Sweden' ]
for i, c in enumerate(nordic_countries):
  print(nordic_countries[i].lower())

denmark
finland
iceland
norway
sweden


In [52]:
enableExports()

     |████████████████████████████████| 92kB 3.5MB/s 
     |████████████████████████████████| 563kB 13.5MB/s 
     |████████████████████████████████| 911kB 17.1MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [88.1 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates In

In [75]:
exportPlots()

Data last updated on 2020-04-17
Data last updated on 2020-04-17
archived_data		     nlargest_multi_plots.json
csse_covid_19_data	     nlargest_multi_plots.png
denmark_country_plots.html   nlargest_multi_plots.svg
denmark_country_plots.json   nordic_multi_plots.html
denmark_country_plots.png    nordic_multi_plots.json
denmark_country_plots.svg    nordic_multi_plots.png
finland_country_plots.html   nordic_multi_plots.svg
finland_country_plots.json   norway_country_plots.html
finland_country_plots.png    norway_country_plots.json
finland_country_plots.svg    norway_country_plots.png
iceland_country_plots.html   norway_country_plots.svg
iceland_country_plots.json   plots.zip
iceland_country_plots.png    README.md
iceland_country_plots.svg    sweden_country_plots.html
nlargest_country_plots.html  sweden_country_plots.json
nlargest_country_plots.json  sweden_country_plots.png
nlargest_country_plots.png   sweden_country_plots.svg
nlargest_country_plots.svg   who_covid_19_situation_reports
nla